# Generate a SQLite database from existing dataframes

In [2]:
import pandas as pd
import sqlite3

data_dir = '../data'
db_name = f'{data_dir}/nsf_awards.db'

In [5]:
def run_query(db,query):
    with sqlite3.connect(db) as conn:
        df = pd.read_sql_query(query, conn)
    return df

In [2]:
# Read in pickle files
# Main tables
awards_df = pd.read_pickle('awards_df.pkl')
pi_df = pd.read_pickle('pi_df.pkl')
pgm_ele_df = pd.read_pickle('pgm_ele_df.pkl')
pgm_ref_df = pd.read_pickle('pgm_ref_df.pkl')
app_fund_df = pd.read_pickle('app_fund_df.pkl')
oblg_fy_df = pd.read_pickle('oblg_fy_df.pkl')

# Bridge tables
awd_pi_df = pd.read_pickle('awd_pi_df.pkl')
awd_pgm_ele_df = pd.read_pickle('awd_pgm_ele_df.pkl')
awd_pgm_ref_df = pd.read_pickle('awd_pgm_ref_df.pkl')

In [3]:
# Write all of the dataframes to a SQLite database

with sqlite3.connect(db_name) as conn:
    awards_df.to_sql('awards', conn, if_exists='replace', index=False)
    pi_df.to_sql('pi', conn, if_exists='replace', index=False)
    pgm_ele_df.to_sql('pgm_ele', conn, if_exists='replace', index=False)
    pgm_ref_df.to_sql('pgm_ref', conn, if_exists='replace', index=False)
    app_fund_df.to_sql('app_fund', conn, if_exists='replace', index=False)
    oblg_fy_df.to_sql('oblg_fy', conn, if_exists='replace', index=False)

    awd_pi_df.to_sql('awd_pi', conn, if_exists='replace', index=False)
    awd_pgm_ele_df.to_sql('awd_pgm_ele', conn, if_exists='replace', index=False)
    awd_pgm_ref_df.to_sql('awd_pgm_ref', conn, if_exists='replace', index=False)

In [3]:
# Test the database
with sqlite3.connect(db_name) as conn:
    # Check the tables
    tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
    print(tables)

    # Check the awards table
    awards_check = pd.read_sql_query("SELECT * FROM awards LIMIT 5;", conn)
    print(awards_check)

    # Check the pi table
    pi_check = pd.read_sql_query("SELECT * FROM pi LIMIT 5;", conn)
    print(pi_check)

    # Check the pgm_ele table
    pgm_ele_check = pd.read_sql_query("SELECT * FROM pgm_ele LIMIT 5;", conn)
    print(pgm_ele_check)

    # Check the pgm_ref table
    pgm_ref_check = pd.read_sql_query("SELECT * FROM pgm_ref LIMIT 5;", conn)
    print(pgm_ref_check)



          name
0       awards
1           pi
2      pgm_ele
3      pgm_ref
4     app_fund
5      oblg_fy
6       awd_pi
7  awd_pgm_ele
8  awd_pgm_ref
    awd_id agcy_id   tran_type           awd_istr_txt  \
0  0415302     NSF  CoopAgrmnt  Cooperative Agreement   
1  0731599     NSF       Grant         Standard Grant   
2  0804541     NSF       Grant         Standard Grant   
3  0805878     NSF       Grant         Standard Grant   
4  0805989     NSF       Grant         Standard Grant   

                                        awd_titl_txt cfda_num  org_code  \
0  Advanced Technology Solar Telescope (ATST) Con...   47.049  03020000   
1  Individual - Nomination for Presidential Award...   47.076  11040300   
2  The Geometric Background of biHamiltonian Systems   47.049  03040000   
3  The Semiclassical Limit and Geometric Quantiza...   47.049  03040000   
4  Application of methods of arithmetic geometry ...   47.049  03040000   

     po_phone        po_email po_sign_block_name  ...  \

In [6]:
query = """
SELECT p.pi_full_name, count(*) as num_awards
FROM pi p
JOIN awd_pi ap ON p.nsf_id = ap.nsf_id
GROUP BY p.pi_full_name
ORDER BY num_awards DESC;
"""

# Print the number of awards by PI
print(run_query(db_name, query).head(10))

            pi_full_name  num_awards
0           Nancy R Gray         178
1  J. Ardie Butch Dillen          76
2         Jerene Shaheed          60
3              Wei Zhang          57
4            Sajal K Das          50
5      Terry B Appelgate          48
6    Nicholas G Feamster          45
7       Nicholas R Bates          42
8     Katherine E Bailey          42
9         Roman Lubynsky          41


In [14]:
query = """
SELECT DISTINCT 
    dir_abbr,
    org_dir_long_name,
    div_abbr,
    org_div_long_name
FROM awards
ORDER BY dir_abbr, div_abbr;
"""

# Print the number of awards by PI
display(run_query(db_name, query))
# list(run_query(db_name, query).div_abbr)

,dir_abbr,org_dir_long_name,div_abbr,org_div_long_name
0,BFA,"Office of Budget, Finance, & Award Management",BFA,"Office of Budget, Finance, & Award Management"
1,BFA,"Office of Budget, Finance, & Award Management",DACS,Division of Acquisition & Cooperative Support
2,BFA,"Office of Budget, Finance, & Award Management",DFM,Division of Financial Management
3,BFA,"Office of Budget, Finance, & Award Management",DIAS,Division of Institution & Award Support
4,BFA,"Office of Budget, Finance, & Award Management",DOB,Budget Division
...,...,...,...,...
62,SBE,"Directorate for Social, Behavioral and Economi...",SES,Division of Social and Economic Sciences
63,SBE,"Directorate for Social, Behavioral and Economi...",SMA,SBE Office of Multidisciplinary Activities
64,TIP,"Directorate for Technology, Innovation, and Pa...",ITE,Innovation and Technology Ecosystems
65,TIP,"Directorate for Technology, Innovation, and Pa...",TF,Technology Frontiers
